In [106]:
import urllib.request
import pandas as pd
from pandas import json_normalize
import json
import time

def get_url(r,month):
    url = 'http://comtrade.un.org/api/get?'
    max_num = '100000'
    freq = 'y'
    ps = month
    px = 'HS'
    rg = '1'
    cc = '8486,27,2709' #TOTAL
    fmt = 'json'
    querytype = 'C'

    query_url = url + 'max=' + max_num + '&r=' + r + '&freq=' + freq + '&ps=' + ps + '&px=' + px 
    query_url = query_url + '&rg=' + rg + '&cc=' + cc + '&fmt=' + fmt + '&type=' + querytype 
    #print(query_url)
    print(r)
    
    return query_url

def get_query_df(url):
    #导入url读取返回的json结果
    req=urllib.request.Request(url)
    resp=urllib.request.urlopen(req)
    data=resp.read().decode('utf-8')

    #将结果转化为字典
    data_dict = json.loads(data)
  
    #将结果转化为dataframe
    df = json_normalize(data_dict['dataset'])
    
    return df

In [107]:
import pandas as pd
counrty_pd = pd.DataFrame(country)
counrty_pd.columns = ['country_name']
#获取code类型之间映射
code_book = pd.read_csv('codebook.csv', header=None, sep="	")
code_book.columns = ['country_name','country_code','Label','3','4','5','6','7']
code_book['country_name'] = code_book['country_name'].str.strip()
APEC_country = pd.merge(counrty_pd, code_book, on='country_name',how='inner')
#获取国家名code映射
counrty_code = pd.read_csv('nodes.csv')
#获取APEC国家总信息
APEC_message = pd.merge(counrty_code, APEC_country, on='Label',how='inner')
country_df.columns = ['idnums','Id']
id_APEC = pd.merge(country_df, APEC_message, on='Id',how='inner')['idnums']



In [108]:
timelist = ['2017']
for monthnumber in timelist:
    print(monthnumber)
    alldata = pd.DataFrame(columns=['rtTitle','ptTitle','cmdCode','TradeValue'])
    citynum = 0
    citystr = ''
    count = 0
    for i in id_APEC:
        citynum += 1
        if(citynum < 3):
            citystr += i + ','
            continue
        else:
            citynum = 0
            citystr += i
            query_url = get_url(citystr,monthnumber)
            #print(query_url)
            citystr = ''
            dataresult = get_query_df(query_url)
            if(dataresult.empty is False):
                alldata=pd.concat([alldata, dataresult[['rtTitle','ptTitle','cmdCode','TradeValue']]])
            else:
                count += 1
                print(str(count) + '空')
            #time.sleep(40)
    if(citynum != 0):
        citynum = 0
        citystr = citystr[:-1]
        query_url = get_url(citystr,monthnumber)
        citystr = ''
        dataresult = get_query_df(query_url)
        if(dataresult.empty is False):
            alldata=pd.concat([alldata, dataresult[['rtTitle','ptTitle','cmdCode','TradeValue']]])
        else:
            count += 1
            print(str(count) + '空')

    #合并结果     
    #alldata.to_csv(monthnumber + '_APEC_import_trade.csv', index = False)

2017
36,96,124
152,156,344
360,392,458
484,554,598
604,608,410
643,702,764
842,704


In [109]:
import numpy as np
#读取统一名称
#由于按年份的美国命名为USA，需要特别注意，
APEC_country = pd.read_csv('APEC_country.csv')
APEC_country.iloc[1,4] = 'China, Hong Kong SAR'
#由于按年份的美国命名为USA，需要特别注意，
APEC_country.iloc[18,4] = 'USA'
APEC_country.iloc[9,4] = 'Rep. of Korea'
APEC_country = APEC_country[['un_name','en_name']]
APEC_country.columns=['rtTitle','en_name']

result_name = 'TradeValue'

#国家名映射
result = pd.merge(APEC_country, alldata, on='rtTitle',how='inner')
result = result[['en_name','cmdCode', 'ptTitle', result_name]]
result.columns=['country1', 'cmdCode', 'rtTitle', result_name]
result = pd.merge(APEC_country, result, on='rtTitle',how='inner')
result = result[['cmdCode','country1', 'en_name', result_name]]
result.columns=['cmdCode','country1', 'country2', result_name]

#去掉文莱
country_list = APEC_country['en_name'].unique()
country_list.sort()
country_list = np.delete(country_list, 1)
result_frame = ['country']
for i in country_list:
    result_frame.append(i)
len(result_frame)
#result_pd = pd.DataFrame(columns = result_frame)
#加载货物id
goods_code = alldata['cmdCode'].unique()

In [110]:
result
for good in goods_code:
    result_pd = pd.DataFrame(columns = result_frame)
    for country_A in country_list:
        now_list = []
        now_list.append(country_A)
        for country_B in country_list:
            if(country_A == country_B):
                now_list.append(0)
            elif(result[(result['country1'] == country_A) & (result['country2'] == country_B)& (result['cmdCode'] == good)].empty == 0):
                now_list.append(result[(result['country1'] == country_A) & (result['country2'] == country_B)& (result['cmdCode'] == good)].iloc[0,3])
            else:
                now_list.append(-1)
        #print(len(now_list))
        result_pd.loc[len(result_pd)] = now_list
    #print(time)
    #break
    #print(result_pd)
    result_pd.to_csv( good +'_2017.csv' , index = False)

In [105]:
result_pd

,country,Australia,Canada,Chile,China,Hong Kong,Indonesia,Japan,Malaysia,Mexico,New Zealand,Peru,Philippines,Russia,Singapore,South Korea,Taiwan,Thailand,United States,Vietnam
0,Australia,0,1719364815,368739623,49971610459,590670668,3213593511,16523672763,8632210571,2216844845,6059370800,282981528,525103915,328114231,6377954537,15892262737,-1,11073613826,23612147222,3944661983
1,Canada,1463504389,0,1552221326,54652257046,297204327,1333615050,13500067608,2147328139,27345826480,546869025,1385743248,1072949047,861087243,775171068,6625012968,-1,2544841331,222572824420,3917328251
2,Chile,345999325,767870264,0,15509223265,82122153,209315040,2125951610,206327664,2133121424,130209628,939294463,55223900,67678617,85947264,1955714839,-1,898998293,11751898133,888452453
3,China,95009120929,20423006604,21175532142,0,7317183822,28574305510,165794005814,54426139443,11803239382,9391098199,13367374443,19239177209,41390293056,34249623689,177553154148,-1,41596082905,154441855874,50374616976
4,Hong Kong,8989653742,1650507407,692516276,262582642858,0,2762908289,35714179587,14742730408,841194784,604150408,140700801,11165292776,1010445838,37688513377,32498529891,-1,11684399130,30816433442,7910077586
5,Indonesia,6008949217,1553625816,119896093,35766807500,1837817788,0,15240036557,8858197446,219553059,751181754,63200336,859298194,1302714423,16888495760,8122335641,-1,9281606247,8150185485,3228760343
6,Japan,38908730109,10930632258,6546431005,164543433301,1823815917,19884059785,0,19270956818,5783598493,2477318727,2085745040,9770791387,13822817435,8525859240,28103159813,-1,22732405141,73952984439,18533199404
7,Malaysia,4676735802,707376468,183066168,38279307263,3261911018,8819350622,14788770471,0,326346860,781969698,137445764,2089881883,827576413,21587907638,9101158324,-1,11208591629,15209250216,5267866122
8,Mexico,344117396,9766367332,1536273002,74150192456,331428893,1494273806,18185989885,7886931386,0,357446491,507503931,2361407569,1459152158,1400928711,15756440254,-1,5931486730,194978436628,4615636985
9,New Zealand,4910884426,455847515,83429210,7748410971,63784594,607527324,2960723532,1255400535,238382718,0,31549629,88701249,220603359,1371514051,1421114761,-1,1889662228,4286936568,589906292


In [51]:
result_pd

,country,Australia,Canada,Chile,China,Hong Kong,Indonesia,Japan,Malaysia,Mexico,New Zealand,Peru,Philippines,Russia,Singapore,South Korea,Taiwan,Thailand,United States,Vietnam
0,Australia,0,1719364815,368739623,49971610459,-1,3213593511,16523672763,8632210571,2216844845,6059370800,282981528,525103915,-1,6377954537,-1,-1,11073613826,-1,-1
1,Canada,1463504389,0,1552221326,54652257046,-1,1333615050,13500067608,2147328139,27345826480,546869025,1385743248,1072949047,-1,775171068,-1,-1,2544841331,-1,-1
2,Chile,345999325,767870264,0,15509223265,-1,209315040,2125951610,206327664,2133121424,130209628,939294463,55223900,-1,85947264,-1,-1,898998293,-1,-1
3,China,95009120929,20423006604,21175532142,0,-1,28574305510,165794005814,54426139443,11803239382,9391098199,13367374443,19239177209,-1,34249623689,-1,-1,41596082905,-1,-1
4,Hong Kong,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,Indonesia,6008949217,1553625816,119896093,35766807500,-1,0,15240036557,8858197446,219553059,751181754,63200336,859298194,-1,16888495760,-1,-1,9281606247,-1,-1
6,Japan,38908730109,10930632258,6546431005,164543433301,-1,19884059785,0,19270956818,5783598493,2477318727,2085745040,9770791387,-1,8525859240,-1,-1,22732405141,-1,-1
7,Malaysia,4676735802,707376468,183066168,38279307263,-1,8819350622,14788770471,0,326346860,781969698,137445764,2089881883,-1,21587907638,-1,-1,11208591629,-1,-1
8,Mexico,344117396,9766367332,1536273002,74150192456,-1,1494273806,18185989885,7886931386,0,357446491,507503931,2361407569,-1,1400928711,-1,-1,5931486730,-1,-1
9,New Zealand,4910884426,455847515,83429210,7748410971,-1,607527324,2960723532,1255400535,238382718,0,31549629,88701249,-1,1371514051,-1,-1,1889662228,-1,-1


In [15]:
country_list = """中国
People's Republic of China
中华人民共和国
1991年11月
香港
Hong Kong， China
中华人民共和国香港特别行政区
1991年11月
台湾
ChineseTaipei
中国台北
1991年11月
澳大利亚
Australia
澳大利亚联邦
1989年11月
文莱
Brunei Darussalam
文莱达鲁萨兰国
1989年11月
加拿大
Canada
加拿大
1989年11月
智利
Chile
智利共和国
1994年11月
印尼
Indonesia
印度尼西亚共和国
1989年11月
日本
Japan
日本国
1989年11月
韩国
Republic of Korea
大韩民国
1989年11月
马来西亚
Malaysia
马来西亚
1989年11月
墨西哥
Mexico
墨西哥合众国
1993年11月
新西兰
New Zealand
新西兰
1989年11月
秘鲁
Peru
秘鲁共和国
1998年11月
巴布亚新几内亚
Papua New Guinea
巴布亚新几内亚独立国
1993年11月
菲律宾
The Philippines
菲律宾共和国
1989年11月
俄罗斯
Russia
俄罗斯联邦
1998年11月
新加坡
Singapore
新加坡共和国
1989年11月
泰国
Thailand
泰王国
1989年11月
美国
The United States
美利坚合众国
1989年11月
越南
Viet Nam
越南社会主义共和国
1999年11月"""
country = []
count = 0
for i in country_list.split('\n'):
    if(count%4 == 0):
        country.append(i)
        
    count += 1

In [14]:
countrycode = """{
    "more": false,
    "results": [
        {
            "id": "all",
            "text": "All"
        },
        {
            "id": "4",
            "text": "Afghanistan"
        },
        {
            "id": "8",
            "text": "Albania"
        },
        {
            "id": "12",
            "text": "Algeria"
        },
        {
            "id": "20",
            "text": "Andorra"
        },
        {
            "id": "24",
            "text": "Angola"
        },
        {
            "id": "660",
            "text": "Anguilla"
        },
        {
            "id": "28",
            "text": "Antigua and Barbuda"
        },
        {
            "id": "32",
            "text": "Argentina"
        },
        {
            "id": "51",
            "text": "Armenia"
        },
        {
            "id": "533",
            "text": "Aruba"
        },
        {
            "id": "36",
            "text": "Australia"
        },
        {
            "id": "40",
            "text": "Austria"
        },
        {
            "id": "31",
            "text": "Azerbaijan"
        },
        {
            "id": "44",
            "text": "Bahamas"
        },
        {
            "id": "48",
            "text": "Bahrain"
        },
        {
            "id": "50",
            "text": "Bangladesh"
        },
        {
            "id": "52",
            "text": "Barbados"
        },
        {
            "id": "112",
            "text": "Belarus"
        },
        {
            "id": "56",
            "text": "Belgium"
        },
        {
            "id": "58",
            "text": "Belgium-Luxembourg"
        },
        {
            "id": "84",
            "text": "Belize"
        },
        {
            "id": "204",
            "text": "Benin"
        },
        {
            "id": "60",
            "text": "Bermuda"
        },
        {
            "id": "64",
            "text": "Bhutan"
        },
        {
            "id": "68",
            "text": "Bolivia (Plurinational State of)"
        },
        {
            "id": "535",
            "text": "Bonaire"
        },
        {
            "id": "70",
            "text": "Bosnia Herzegovina"
        },
        {
            "id": "72",
            "text": "Botswana"
        },
        {
            "id": "92",
            "text": "Br. Virgin Isds"
        },
        {
            "id": "76",
            "text": "Brazil"
        },
        {
            "id": "96",
            "text": "Brunei Darussalam"
        },
        {
            "id": "100",
            "text": "Bulgaria"
        },
        {
            "id": "854",
            "text": "Burkina Faso"
        },
        {
            "id": "108",
            "text": "Burundi"
        },
        {
            "id": "132",
            "text": "Cabo Verde"
        },
        {
            "id": "116",
            "text": "Cambodia"
        },
        {
            "id": "120",
            "text": "Cameroon"
        },
        {
            "id": "124",
            "text": "Canada"
        },
        {
            "id": "136",
            "text": "Cayman Isds"
        },
        {
            "id": "140",
            "text": "Central African Rep."
        },
        {
            "id": "148",
            "text": "Chad"
        },
        {
            "id": "152",
            "text": "Chile"
        },
        {
            "id": "156",
            "text": "China"
        },
        {
            "id": "344",
            "text": "China, Hong Kong SAR"
        },
        {
            "id": "446",
            "text": "China, Macao SAR"
        },
        {
            "id": "170",
            "text": "Colombia"
        },
        {
            "id": "174",
            "text": "Comoros"
        },
        {
            "id": "178",
            "text": "Congo"
        },
        {
            "id": "184",
            "text": "Cook Isds"
        },
        {
            "id": "188",
            "text": "Costa Rica"
        },
        {
            "id": "384",
            "text": "Côte d'Ivoire"
        },
        {
            "id": "191",
            "text": "Croatia"
        },
        {
            "id": "192",
            "text": "Cuba"
        },
        {
            "id": "531",
            "text": "Curaçao"
        },
        {
            "id": "196",
            "text": "Cyprus"
        },
        {
            "id": "203",
            "text": "Czechia"
        },
        {
            "id": "200",
            "text": "Czechoslovakia"
        },
        {
            "id": "408",
            "text": "Dem. People's Rep. of Korea"
        },
        {
            "id": "180",
            "text": "Dem. Rep. of the Congo"
        },
        {
            "id": "208",
            "text": "Denmark"
        },
        {
            "id": "262",
            "text": "Djibouti"
        },
        {
            "id": "212",
            "text": "Dominica"
        },
        {
            "id": "214",
            "text": "Dominican Rep."
        },
        {
            "id": "588",
            "text": "East and West Pakistan"
        },
        {
            "id": "218",
            "text": "Ecuador"
        },
        {
            "id": "818",
            "text": "Egypt"
        },
        {
            "id": "222",
            "text": "El Salvador"
        },
        {
            "id": "226",
            "text": "Equatorial Guinea"
        },
        {
            "id": "232",
            "text": "Eritrea"
        },
        {
            "id": "233",
            "text": "Estonia"
        },
        {
            "id": "231",
            "text": "Ethiopia"
        },
        {
            "id": "97",
            "text": "EU"
        },
        {
            "id": "234",
            "text": "Faeroe Isds"
        },
        {
            "id": "238",
            "text": "Falkland Isds (Malvinas)"
        },
        {
            "id": "242",
            "text": "Fiji"
        },
        {
            "id": "246",
            "text": "Finland"
        },
        {
            "id": "886",
            "text": "Fmr Arab Rep. of Yemen"
        },
        {
            "id": "278",
            "text": "Fmr Dem. Rep. of Germany"
        },
        {
            "id": "866",
            "text": "Fmr Dem. Rep. of Vietnam"
        },
        {
            "id": "720",
            "text": "Fmr Dem. Yemen"
        },
        {
            "id": "230",
            "text": "Fmr Ethiopia"
        },
        {
            "id": "280",
            "text": "Fmr Fed. Rep. of Germany"
        },
        {
            "id": "582",
            "text": "Fmr Pacific Isds"
        },
        {
            "id": "590",
            "text": "Fmr Panama, excl.Canal Zone"
        },
        {
            "id": "592",
            "text": "Fmr Panama-Canal-Zone"
        },
        {
            "id": "868",
            "text": "Fmr Rep. of Vietnam"
        },
        {
            "id": "717",
            "text": "Fmr Rhodesia Nyas"
        },
        {
            "id": "736",
            "text": "Fmr Sudan"
        },
        {
            "id": "835",
            "text": "Fmr Tanganyika"
        },
        {
            "id": "810",
            "text": "Fmr USSR"
        },
        {
            "id": "890",
            "text": "Fmr Yugoslavia"
        },
        {
            "id": "836",
            "text": "Fmr Zanzibar and Pemba Isd"
        },
        {
            "id": "251",
            "text": "France"
        },
        {
            "id": "254",
            "text": "French Guiana"
        },
        {
            "id": "258",
            "text": "French Polynesia"
        },
        {
            "id": "583",
            "text": "FS Micronesia"
        },
        {
            "id": "266",
            "text": "Gabon"
        },
        {
            "id": "270",
            "text": "Gambia"
        },
        {
            "id": "268",
            "text": "Georgia"
        },
        {
            "id": "276",
            "text": "Germany"
        },
        {
            "id": "288",
            "text": "Ghana"
        },
        {
            "id": "292",
            "text": "Gibraltar"
        },
        {
            "id": "300",
            "text": "Greece"
        },
        {
            "id": "304",
            "text": "Greenland"
        },
        {
            "id": "308",
            "text": "Grenada"
        },
        {
            "id": "312",
            "text": "Guadeloupe"
        },
        {
            "id": "320",
            "text": "Guatemala"
        },
        {
            "id": "324",
            "text": "Guinea"
        },
        {
            "id": "624",
            "text": "Guinea-Bissau"
        },
        {
            "id": "328",
            "text": "Guyana"
        },
        {
            "id": "332",
            "text": "Haiti"
        },
        {
            "id": "336",
            "text": "Holy See (Vatican City State)"
        },
        {
            "id": "340",
            "text": "Honduras"
        },
        {
            "id": "348",
            "text": "Hungary"
        },
        {
            "id": "352",
            "text": "Iceland"
        },
        {
            "id": "699",
            "text": "India"
        },
        {
            "id": "356",
            "text": "India, excl. Sikkim"
        },
        {
            "id": "360",
            "text": "Indonesia"
        },
        {
            "id": "364",
            "text": "Iran"
        },
        {
            "id": "368",
            "text": "Iraq"
        },
        {
            "id": "372",
            "text": "Ireland"
        },
        {
            "id": "376",
            "text": "Israel"
        },
        {
            "id": "381",
            "text": "Italy"
        },
        {
            "id": "388",
            "text": "Jamaica"
        },
        {
            "id": "392",
            "text": "Japan"
        },
        {
            "id": "400",
            "text": "Jordan"
        },
        {
            "id": "398",
            "text": "Kazakhstan"
        },
        {
            "id": "404",
            "text": "Kenya"
        },
        {
            "id": "296",
            "text": "Kiribati"
        },
        {
            "id": "414",
            "text": "Kuwait"
        },
        {
            "id": "417",
            "text": "Kyrgyzstan"
        },
        {
            "id": "418",
            "text": "Lao People's Dem. Rep."
        },
        {
            "id": "428",
            "text": "Latvia"
        },
        {
            "id": "422",
            "text": "Lebanon"
        },
        {
            "id": "426",
            "text": "Lesotho"
        },
        {
            "id": "430",
            "text": "Liberia"
        },
        {
            "id": "434",
            "text": "Libya"
        },
        {
            "id": "440",
            "text": "Lithuania"
        },
        {
            "id": "442",
            "text": "Luxembourg"
        },
        {
            "id": "450",
            "text": "Madagascar"
        },
        {
            "id": "454",
            "text": "Malawi"
        },
        {
            "id": "458",
            "text": "Malaysia"
        },
        {
            "id": "462",
            "text": "Maldives"
        },
        {
            "id": "466",
            "text": "Mali"
        },
        {
            "id": "470",
            "text": "Malta"
        },
        {
            "id": "584",
            "text": "Marshall Isds"
        },
        {
            "id": "474",
            "text": "Martinique"
        },
        {
            "id": "478",
            "text": "Mauritania"
        },
        {
            "id": "480",
            "text": "Mauritius"
        },
        {
            "id": "175",
            "text": "Mayotte"
        },
        {
            "id": "484",
            "text": "Mexico"
        },
        {
            "id": "496",
            "text": "Mongolia"
        },
        {
            "id": "499",
            "text": "Montenegro"
        },
        {
            "id": "500",
            "text": "Montserrat"
        },
        {
            "id": "504",
            "text": "Morocco"
        },
        {
            "id": "508",
            "text": "Mozambique"
        },
        {
            "id": "104",
            "text": "Myanmar"
        },
        {
            "id": "580",
            "text": "N. Mariana Isds"
        },
        {
            "id": "516",
            "text": "Namibia"
        },
        {
            "id": "524",
            "text": "Nepal"
        },
        {
            "id": "530",
            "text": "Neth. Antilles"
        },
        {
            "id": "532",
            "text": "Neth. Antilles and Aruba"
        },
        {
            "id": "528",
            "text": "Netherlands"
        },
        {
            "id": "540",
            "text": "New Caledonia"
        },
        {
            "id": "554",
            "text": "New Zealand"
        },
        {
            "id": "558",
            "text": "Nicaragua"
        },
        {
            "id": "562",
            "text": "Niger"
        },
        {
            "id": "566",
            "text": "Nigeria"
        },
        {
            "id": "579",
            "text": "Norway"
        },
        {
            "id": "512",
            "text": "Oman"
        },
        {
            "id": "490",
            "text": "Other Asia, nes"
        },
        {
            "id": "586",
            "text": "Pakistan"
        },
        {
            "id": "585",
            "text": "Palau"
        },
        {
            "id": "591",
            "text": "Panama"
        },
        {
            "id": "598",
            "text": "Papua New Guinea"
        },
        {
            "id": "600",
            "text": "Paraguay"
        },
        {
            "id": "459",
            "text": "Peninsula Malaysia"
        },
        {
            "id": "604",
            "text": "Peru"
        },
        {
            "id": "608",
            "text": "Philippines"
        },
        {
            "id": "616",
            "text": "Poland"
        },
        {
            "id": "620",
            "text": "Portugal"
        },
        {
            "id": "634",
            "text": "Qatar"
        },
        {
            "id": "410",
            "text": "Rep. of Korea"
        },
        {
            "id": "498",
            "text": "Rep. of Moldova"
        },
        {
            "id": "638",
            "text": "Réunion"
        },
        {
            "id": "642",
            "text": "Romania"
        },
        {
            "id": "643",
            "text": "Russian Federation"
        },
        {
            "id": "646",
            "text": "Rwanda"
        },
        {
            "id": "647",
            "text": "Ryukyu Isd"
        },
        {
            "id": "461",
            "text": "Sabah"
        },
        {
            "id": "652",
            "text": "Saint Barthelemy"
        },
        {
            "id": "654",
            "text": "Saint Helena"
        },
        {
            "id": "659",
            "text": "Saint Kitts and Nevis"
        },
        {
            "id": "658",
            "text": "Saint Kitts, Nevis and Anguilla"
        },
        {
            "id": "662",
            "text": "Saint Lucia"
        },
        {
            "id": "534",
            "text": "Saint Maarten"
        },
        {
            "id": "666",
            "text": "Saint Pierre and Miquelon"
        },
        {
            "id": "670",
            "text": "Saint Vincent and the Grenadines"
        },
        {
            "id": "882",
            "text": "Samoa"
        },
        {
            "id": "674",
            "text": "San Marino"
        },
        {
            "id": "678",
            "text": "Sao Tome and Principe"
        },
        {
            "id": "457",
            "text": "Sarawak"
        },
        {
            "id": "682",
            "text": "Saudi Arabia"
        },
        {
            "id": "686",
            "text": "Senegal"
        },
        {
            "id": "688",
            "text": "Serbia"
        },
        {
            "id": "891",
            "text": "Serbia and Montenegro"
        },
        {
            "id": "690",
            "text": "Seychelles"
        },
        {
            "id": "694",
            "text": "Sierra Leone"
        },
        {
            "id": "702",
            "text": "Singapore"
        },
        {
            "id": "703",
            "text": "Slovakia"
        },
        {
            "id": "705",
            "text": "Slovenia"
        },
        {
            "id": "711",
            "text": "So. African Customs Union"
        },
        {
            "id": "90",
            "text": "Solomon Isds"
        },
        {
            "id": "706",
            "text": "Somalia"
        },
        {
            "id": "710",
            "text": "South Africa"
        },
        {
            "id": "728",
            "text": "South Sudan"
        },
        {
            "id": "724",
            "text": "Spain"
        },
        {
            "id": "144",
            "text": "Sri Lanka"
        },
        {
            "id": "275",
            "text": "State of Palestine"
        },
        {
            "id": "729",
            "text": "Sudan"
        },
        {
            "id": "740",
            "text": "Suriname"
        },
        {
            "id": "748",
            "text": "Eswatini"
        },
        {
            "id": "752",
            "text": "Sweden"
        },
        {
            "id": "757",
            "text": "Switzerland"
        },
        {
            "id": "760",
            "text": "Syria"
        },
        {
            "id": "762",
            "text": "Tajikistan"
        },
        {
            "id": "807",
            "text": "North Macedonia"
        },
        {
            "id": "764",
            "text": "Thailand"
        },
        {
            "id": "626",
            "text": "Timor-Leste"
        },
        {
            "id": "768",
            "text": "Togo"
        },
        {
            "id": "772",
            "text": "Tokelau"
        },
        {
            "id": "776",
            "text": "Tonga"
        },
        {
            "id": "780",
            "text": "Trinidad and Tobago"
        },
        {
            "id": "788",
            "text": "Tunisia"
        },
        {
            "id": "792",
            "text": "Turkey"
        },
        {
            "id": "795",
            "text": "Turkmenistan"
        },
        {
            "id": "796",
            "text": "Turks and Caicos Isds"
        },
        {
            "id": "798",
            "text": "Tuvalu"
        },
        {
            "id": "800",
            "text": "Uganda"
        },
        {
            "id": "804",
            "text": "Ukraine"
        },
        {
            "id": "784",
            "text": "United Arab Emirates"
        },
        {
            "id": "826",
            "text": "United Kingdom"
        },
        {
            "id": "834",
            "text": "United Rep. of Tanzania"
        },
        {
            "id": "858",
            "text": "Uruguay"
        },
        {
            "id": "850",
            "text": "US Virgin Isds"
        },
        {
            "id": "842",
            "text": "USA"
        },
        {
            "id": "841",
            "text": "USA (before 1981)"
        },
        {
            "id": "860",
            "text": "Uzbekistan"
        },
        {
            "id": "548",
            "text": "Vanuatu"
        },
        {
            "id": "862",
            "text": "Venezuela"
        },
        {
            "id": "704",
            "text": "Viet Nam"
        },
        {
            "id": "876",
            "text": "Wallis and Futuna Isds"
        },
        {
            "id": "887",
            "text": "Yemen"
        },
        {
            "id": "894",
            "text": "Zambia"
        },
        {
            "id": "716",
            "text": "Zimbabwe"
        },
        {
            "id": "975",
            "text": "ASEAN"
        }
    ]
}"""
country_dict = json.loads(countrycode)
country_df = json_normalize(country_dict['results'])